In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import multivariate_normal as mvn

from armored.models import *
from armored.preprocessing import *

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold

# set plot parameters
params = {'legend.fontsize': 18,
          'figure.figsize': (16, 12),
          'lines.linewidth': 4,
          'axes.labelsize': 24,
          'axes.titlesize':24,
          'axes.linewidth':5,
          'xtick.labelsize':20,
          'ytick.labelsize':20}
plt.rcParams.update(params)
plt.style.use('seaborn-colorblind')
plt.rcParams['pdf.fonttype'] = 42

np.random.seed(123)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-xl5m6r1g because the default path (/home/jaron/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# Set parameters of script

In [2]:
# define k for k-fold (n_splits = k)
n_splits = 20

# number of times to run k-fold x-validation 
# each trial uses different random partitioning of data
n_trials = 10

# import data

In [3]:
df = pd.read_csv("Data/2021_02_19_MultifunctionalDynamicData.csv")

all_experiments = df.Experiments.values
unique_experiments = np.unique(all_experiments)

df.head()

,Experiments,Time,PC_OD,PJ_OD,BV_OD,BF_OD,BO_OD,BT_OD,BC_OD,BY_OD,...,CG_OD,ER_OD,RI_OD,CC_OD,DL_OD,DF_OD,Butyrate,Acetate,Lactate,Succinate
0,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,0,0.000471,0.0,0.000471,0.000000,0.000000,0.000471,0.000471,0.0,...,0.000471,0.000471,0.000471,0.0,0.0,0.000471,0.000000,0.000000,28.000000,0.000000
1,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,16,0.465116,0.0,0.029207,0.000000,0.000000,0.249717,0.500651,0.0,...,0.024339,0.327601,0.001460,0.0,0.0,0.392830,23.092697,47.849302,18.910852,26.141885
2,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,32,0.104523,0.0,0.027928,0.000000,0.000000,0.220107,0.380210,0.0,...,0.020739,0.293384,0.000830,0.0,0.0,0.280111,23.996267,38.915218,17.977137,26.884748
3,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,48,0.124852,0.0,0.012194,0.000000,0.000000,0.268268,0.333970,0.0,...,0.023660,0.188188,0.002366,0.0,0.0,0.224952,24.839219,34.325914,19.406971,31.628061
4,BV-BF-BO-BT-BU-DP-BL-BA-BP-EL-FP-CH-AC-BH-CG-E...,0,0.000000,0.0,0.000388,0.000388,0.000388,0.000388,0.000000,0.0,...,0.000388,0.000388,0.000388,0.0,0.0,0.000000,0.000000,0.000000,28.000000,0.000000


### Define which variables are species and which are metabolites

In [4]:
# specify species and metabolite names 
species = df.columns.values[2:-4]
metabolites =  df.columns.values[-4:]
controls = []
system_variables = np.concatenate((np.array(species), np.array(metabolites)))
system_variables

array(['PC_OD', 'PJ_OD', 'BV_OD', 'BF_OD', 'BO_OD', 'BT_OD', 'BC_OD',
       'BY_OD', 'BU_OD', 'DP_OD', 'BL_OD', 'BA_OD', 'BP_OD', 'CA_OD',
       'EL_OD', 'FP_OD', 'CH_OD', 'AC_OD', 'BH_OD', 'CG_OD', 'ER_OD',
       'RI_OD', 'CC_OD', 'DL_OD', 'DF_OD', 'Butyrate', 'Acetate',
       'Lactate', 'Succinate'], dtype=object)

In [5]:
# perform k-fold x-validation several times (n_trials) with randomized partitions of data
for k in range(n_trials):
    
    # initialize dataframe to save predictions
    df_kfold = pd.DataFrame()

    # object to get indices of kfold splits
    kf = KFold(n_splits=n_splits, random_state=k, shuffle=True)

    # loop over k-fold splits
    for train_index, test_index in kf.split(unique_experiments):

        # update index values to pull trajectories 
        train_index = np.in1d(all_experiments, unique_experiments[train_index])
        test_index = np.in1d(all_experiments, unique_experiments[test_index])

        # pull train and test indices
        train_df  = df.iloc[train_index].copy() 
        test_df   = df.iloc[test_index].copy() 

        # format data
        train_data, exp_names, N = format_data(train_df, species, metabolites, controls)
        test_data, test_exp_names, _ = format_data(test_df, species, metabolites, controls)

        # transform data
        scaler = ZeroMaxScaler().fit(train_data)
        train_data = scaler.transform(train_data)
        test_data  = scaler.transform(test_data)

        # Apply RNN to predict metabolite concentration on sparsely sampled data:
        brnn = RNN(n_species=len(species), n_metabolites=len(metabolites), n_controls=0, 
                     n_hidden=16, N=N, alpha_0=1.)
        brnn.fit(train_data)

        # Make predictions
        preds, stdvs, COV = brnn.predict(test_data)

        # Transform values back to original scale
        preds = scaler.inverse_transform(preds)
        stdvs = scaler.inverse_transform_stdv(stdvs)
        test_data = scaler.inverse_transform(test_data)

        # save data
        pred_df_kfold = pd.DataFrame()
        for i, exp_name in enumerate(test_exp_names):
            pred_df = pd.DataFrame()
            pred_df['Experiments'] = [exp_name]*4
            pred_df['Time'] = [0, 16, 32, 48]
            for j, variable in enumerate(system_variables):
                pred_df[variable + " true"] = test_data[i, :, j]
                pred_df[variable + " pred"] = preds[i, :, j]
                pred_df[variable + " stdv"] = stdvs[i, :, j]
            pred_df_kfold = pd.concat((pred_df_kfold, pred_df))
        df_kfold = pd.concat((df_kfold, pred_df_kfold))
        
    # save kfold predictions
    df_kfold.to_csv(f"Fig2/Kfold/RNN_{n_splits}_fold_{k+1}.csv", index=False)

Epoch: 0, Loss: 1091.99009, Residuals: -3.81530, Convergence:   inf
Epoch: 10, Loss: 1026.99622, Residuals: -3.46652, Convergence: 0.006959
Epoch: 20, Loss: 948.05377, Residuals: -3.15255, Convergence: 0.008889
Epoch: 30, Loss: 861.96710, Residuals: -2.86391, Convergence: 0.009915
Epoch: 40, Loss: 780.60071, Residuals: -2.60227, Convergence: 0.009877
Epoch: 50, Loss: 709.72388, Residuals: -2.36523, Convergence: 0.009275
Epoch: 60, Loss: 649.69806, Residuals: -2.15069, Convergence: 0.008520
Epoch: 70, Loss: 599.44647, Residuals: -1.95706, Convergence: 0.007728
Epoch: 80, Loss: 557.45471, Residuals: -1.78249, Convergence: 0.006922
Epoch: 90, Loss: 522.50250, Residuals: -1.62434, Convergence: 0.006141
Epoch: 100, Loss: 493.42535, Residuals: -1.48119, Convergence: 0.005413
Epoch: 110, Loss: 469.17975, Residuals: -1.35159, Convergence: 0.004758
Epoch: 120, Loss: 448.93692, Residuals: -1.23449, Convergence: 0.004140
Epoch: 130, Loss: 432.02469, Residuals: -1.12917, Convergence: 0.003622
Epoc

Epoch: 200, Loss: 372.52054, Residuals: -0.60715, Convergence: 0.001336
Epoch: 210, Loss: 367.87234, Residuals: -0.55894, Convergence: 0.001180
Epoch: 220, Loss: 363.86832, Residuals: -0.51562, Convergence: 0.001029

Epoch: 221, Evidence: 589.83643, Convergence:   inf

Epoch: 223, Evidence: 2484.51514, Convergence: 0.762595
Epoch: 230, Loss: 1280.18079, Residuals: -0.46899, Convergence: 0.001373
Epoch: 240, Loss: 1264.46216, Residuals: -0.42561, Convergence: 0.001128

Epoch: 248, Evidence: 3375.08203, Convergence: 0.263865
Epoch: 250, Loss: 1591.66870, Residuals: -0.38644, Convergence: 0.001438
Epoch: 260, Loss: 1571.09924, Residuals: -0.35087, Convergence: 0.001196
Epoch: 270, Loss: 1554.07007, Residuals: -0.31867, Convergence: 0.001007

Epoch: 271, Evidence: 3766.29541, Convergence: 0.103872
Epoch: 280, Loss: 1748.84949, Residuals: -0.28963, Convergence: 0.001110

Epoch: 287, Evidence: 3933.12231, Convergence: 0.042416
Epoch: 290, Loss: 1852.28467, Residuals: -0.26335, Convergence: 0

Epoch: 100, Loss: 504.83231, Residuals: -1.46550, Convergence: 0.005317
Epoch: 110, Loss: 480.38626, Residuals: -1.33542, Convergence: 0.004693
Epoch: 120, Loss: 459.90247, Residuals: -1.21809, Convergence: 0.004132
Epoch: 130, Loss: 442.65649, Residuals: -1.11220, Convergence: 0.003601
Epoch: 140, Loss: 428.22394, Residuals: -1.01688, Convergence: 0.003092
Epoch: 150, Loss: 416.13019, Residuals: -0.93095, Convergence: 0.002690
Epoch: 160, Loss: 405.98770, Residuals: -0.85365, Convergence: 0.002323
Epoch: 170, Loss: 397.42136, Residuals: -0.78397, Convergence: 0.001983
Epoch: 180, Loss: 390.16501, Residuals: -0.72126, Convergence: 0.001717
Epoch: 190, Loss: 384.06158, Residuals: -0.66463, Convergence: 0.001447
Epoch: 200, Loss: 378.87625, Residuals: -0.61359, Convergence: 0.001279
Epoch: 210, Loss: 374.43701, Residuals: -0.56759, Convergence: 0.001116

Epoch: 218, Evidence: 656.88220, Convergence:   inf
Epoch: 220, Loss: 839.17273, Residuals: -0.52452, Convergence: 0.001027

Epoch: 222

Epoch: 220, Loss: 843.90479, Residuals: -0.52010, Convergence: 0.000990

Epoch: 220, Evidence: 2452.41943, Convergence: 0.748813
Epoch: 220, Loss: 1292.00024, Residuals: -0.52010, Convergence:   inf
Epoch: 230, Loss: 1273.72253, Residuals: -0.47196, Convergence: 0.001308
Epoch: 240, Loss: 1258.55847, Residuals: -0.42848, Convergence: 0.001103

Epoch: 245, Evidence: 3338.29688, Convergence: 0.265368
Epoch: 250, Loss: 1581.12329, Residuals: -0.38903, Convergence: 0.001356
Epoch: 260, Loss: 1561.61487, Residuals: -0.35323, Convergence: 0.001149

Epoch: 268, Evidence: 3737.38916, Convergence: 0.106784
Epoch: 270, Loss: 1761.47742, Residuals: -0.32083, Convergence: 0.001270
Epoch: 280, Loss: 1741.27271, Residuals: -0.29151, Convergence: 0.001060

Epoch: 284, Evidence: 3909.50049, Convergence: 0.044024
Epoch: 290, Loss: 1847.17358, Residuals: -0.26501, Convergence: 0.001067

Epoch: 294, Evidence: 3985.78198, Convergence: 0.019138
Epoch: 300, Loss: 1897.40405, Residuals: -0.24099, Convergence

Epoch: 260, Loss: 1558.38904, Residuals: -0.36006, Convergence: 0.001196
Epoch: 270, Loss: 1541.49219, Residuals: -0.32745, Convergence: 0.001011

Epoch: 271, Evidence: 3751.86646, Convergence: 0.106717
Epoch: 280, Loss: 1738.43628, Residuals: -0.29794, Convergence: 0.001155

Epoch: 289, Evidence: 3929.55225, Convergence: 0.045218
Epoch: 290, Loss: 1847.08313, Residuals: -0.27116, Convergence: 0.001187
Epoch: 300, Loss: 1827.42236, Residuals: -0.24703, Convergence: 0.000973

Epoch: 300, Evidence: 4012.27051, Convergence: 0.020616
Epoch: 300, Loss: 1899.32092, Residuals: -0.24703, Convergence:   inf

Epoch: 307, Evidence: 4054.74512, Convergence: 0.010475
Epoch: 310, Loss: 1921.16663, Residuals: -0.22515, Convergence: 0.001035

Epoch: 313, Evidence: 4079.66504, Convergence: 0.006108

Epoch: 317, Evidence: 4095.48755, Convergence: 0.003863

Epoch: 319, Evidence: 4103.14941, Convergence: 0.001867
Epoch: 320, Loss: 1956.67737, Residuals: -0.20532, Convergence: 0.001009

Epoch: 322, Evidenc

...


Epoch: 298, Evidence: 3966.02271, Convergence: 0.020162
Epoch: 300, Loss: 1903.05493, Residuals: -0.27006, Convergence: 0.001086

Epoch: 305, Evidence: 4007.26880, Convergence: 0.010293
Epoch: 310, Loss: 1924.41309, Residuals: -0.24557, Convergence: 0.000976

Epoch: 310, Evidence: 4031.08691, Convergence: 0.005909
Epoch: 310, Loss: 1949.87817, Residuals: -0.24557, Convergence:   inf

Epoch: 313, Evidence: 4044.29126, Convergence: 0.003265

Epoch: 316, Evidence: 4054.17627, Convergence: 0.002438

Epoch: 318, Evidence: 4060.51758, Convergence: 0.001562

Epoch: 319, Evidence: 4064.14551, Convergence: 0.000893
Epoch: 320, Loss: 1965.50439, Residuals: -0.22109, Convergence: 0.001935
Epoch: 330, Loss: 1933.85400, Residuals: -0.18290, Convergence: 0.001385
Epoch: 340, Loss: 1911.63745, Residuals: -0.15131, Convergence: 0.000965

Epoch: 340, Evidence: 4115.51270, Convergence: 0.013363
Epoch: 340, Loss: 1948.01721, Residuals: -0.15131, Convergence:   inf
Epoch: 350, Loss: 1925.34644, Residuals


Epoch: 312, Evidence: 4171.32324, Convergence: 0.001483

Epoch: 314, Evidence: 4176.99414, Convergence: 0.001358

Epoch: 315, Evidence: 4179.47998, Convergence: 0.000595
Epoch: 320, Loss: 1979.08875, Residuals: -0.18757, Convergence: 0.001718
Epoch: 330, Loss: 1951.07043, Residuals: -0.15696, Convergence: 0.001245

Epoch: 338, Evidence: 4235.77344, Convergence: 0.013877
Epoch: 340, Loss: 1966.68591, Residuals: -0.13199, Convergence: 0.001325

Epoch: 348, Evidence: 4266.42871, Convergence: 0.007185
Epoch: 350, Loss: 1971.21045, Residuals: -0.11124, Convergence: 0.001150

Epoch: 355, Evidence: 4285.10547, Convergence: 0.004359
Epoch: 360, Loss: 1968.95898, Residuals: -0.09400, Convergence: 0.000983

Epoch: 360, Evidence: 4298.65430, Convergence: 0.003152
Epoch: 360, Loss: 1982.22595, Residuals: -0.09400, Convergence:   inf

Epoch: 364, Evidence: 4308.49902, Convergence: 0.002285

Epoch: 366, Evidence: 4313.86230, Convergence: 0.001243

Epoch: 368, Evidence: 4319.35449, Convergence: 0.00

Epoch: 360, Loss: 1937.63477, Residuals: -0.11083, Convergence: 0.001024

Epoch: 361, Evidence: 4261.18359, Convergence: 0.007557

Epoch: 368, Evidence: 4280.27393, Convergence: 0.004460
Epoch: 370, Loss: 1961.07678, Residuals: -0.09308, Convergence: 0.001148

Epoch: 374, Evidence: 4294.79492, Convergence: 0.003381

Epoch: 378, Evidence: 4305.57227, Convergence: 0.002503
Epoch: 380, Loss: 1965.51147, Residuals: -0.07805, Convergence: 0.001049

Epoch: 381, Evidence: 4312.24658, Convergence: 0.001548

Epoch: 383, Evidence: 4317.82324, Convergence: 0.001292

Epoch: 385, Evidence: 4321.82031, Convergence: 0.000925
Epoch: 390, Loss: 1954.22742, Residuals: -0.06038, Convergence: 0.001558

Epoch: 397, Evidence: 4348.58838, Convergence: 0.007075
Epoch: 400, Loss: 1947.94775, Residuals: -0.04247, Convergence: 0.001290

Epoch: 405, Evidence: 4371.83691, Convergence: 0.005318
Epoch: 410, Loss: 1942.76514, Residuals: -0.02949, Convergence: 0.001059

Epoch: 411, Evidence: 4388.34912, Convergence: 0


Epoch: 419, Evidence: 4429.61230, Convergence: 0.001802
Epoch: 420, Loss: 1963.07654, Residuals: -0.01932, Convergence: 0.001114

Epoch: 423, Evidence: 4437.72754, Convergence: 0.001829

Epoch: 425, Evidence: 4442.82324, Convergence: 0.001147

Epoch: 428, Evidence: 4449.02734, Convergence: 0.001394
Epoch: 430, Loss: 1960.34192, Residuals: -0.01591, Convergence: 0.000980

Epoch: 430, Evidence: 4453.68164, Convergence: 0.001045
Epoch: 430, Loss: 1964.60034, Residuals: -0.01591, Convergence:   inf

Epoch: 432, Evidence: 4457.20557, Convergence: 0.000791
Epoch: 440, Loss: 1939.59143, Residuals: -0.01183, Convergence: 0.000982

Epoch: 440, Evidence: 4476.27197, Convergence: 0.005047
Epoch: 440, Loss: 1953.33875, Residuals: -0.01183, Convergence:   inf

Epoch: 446, Evidence: 4494.10547, Convergence: 0.003968
Epoch: 450, Loss: 1941.92554, Residuals: -0.01096, Convergence: 0.000911

Epoch: 450, Evidence: 4506.47070, Convergence: 0.002744
Epoch: 450, Loss: 1951.55762, Residuals: -0.01096, Conv

Epoch: 430, Loss: 1939.73669, Residuals: -0.03088, Convergence: 0.001462

Epoch: 434, Evidence: 4425.36914, Convergence: 0.002353

Epoch: 438, Evidence: 4435.37158, Convergence: 0.002255
Epoch: 440, Loss: 1933.77148, Residuals: -0.03442, Convergence: 0.001241

Epoch: 444, Evidence: 4445.01904, Convergence: 0.002170
Epoch: 450, Loss: 1920.15552, Residuals: -0.03803, Convergence: 0.000878

Epoch: 450, Evidence: 4458.36133, Convergence: 0.002993
Epoch: 450, Loss: 1931.71338, Residuals: -0.03803, Convergence:   inf

Epoch: 454, Evidence: 4467.75293, Convergence: 0.002102

Epoch: 457, Evidence: 4474.13916, Convergence: 0.001427
Epoch: 460, Loss: 1924.71069, Residuals: -0.04308, Convergence: 0.000918

Epoch: 460, Evidence: 4478.08203, Convergence: 0.000880
Epoch: 460, Loss: 1927.55920, Residuals: -0.04308, Convergence:   inf

Epoch: 463, Evidence: 4478.07812, Convergence: 0.000880

Epoch: 465, Evidence: 4478.04395, Convergence: 0.000872
Epoch: 0, Loss: 1109.12718, Residuals: -3.85992, Conver